In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Maskeleri çıkartma

In [2]:
import os
import cv2
from google.colab import drive

drive.mount('/content/drive')

input_dir = '/content/drive/MyDrive/wood_otsu/wood/ground_truth/defect'
output_dir = '/content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect'

os.makedirs(output_dir, exist_ok=True)

target_size = (256, 256)

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"⚠️ {filename} yüklenemedi.")
            continue

        resized = cv2.resize(img, target_size, interpolation=cv2.INTER_NEAREST)

        save_path = os.path.join(output_dir, filename)
        cv2.imwrite(save_path, resized)
        print(f"✅ Kaydedildi: {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000006_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000008_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000000_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000007_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000003_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/101900010_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100700054_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/101900004_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/

# Yeni Klasöre Oluşturup Veri Kullanımını Kolaylaştırmak

In [26]:
import os
import shutil

# Kaynak klasör yolu
source_dir = '/content/drive/MyDrive/wood_otsu'

# Hedef klasör yolu
destination_dir = '/content/drive/MyDrive/wood_otsu_augmentation'

# Hedef klasörü oluştur (eğer mevcut değilse)
os.makedirs(destination_dir, exist_ok=True)

# Kaynak klasöründeki tüm klasörleri ve dosyaları yinele
for root, dirs, files in os.walk(source_dir):
    # Her klasör için
    for dir in dirs:
        # Kaynak klasör yolu
        source_subdir = os.path.join(root, dir)

        # Hedef klasör yolu (wood_otsu_augmentation altında aynı klasör yapısı)
        relative_path = os.path.relpath(source_subdir, source_dir)
        destination_subdir = os.path.join(destination_dir, relative_path)

        # Hedef alt klasörü oluştur
        os.makedirs(destination_subdir, exist_ok=True)

        # Alt klasördeki tüm dosyaları kopyala
        for filename in os.listdir(source_subdir):
            source_file_path = os.path.join(source_subdir, filename)
            destination_file_path = os.path.join(destination_subdir, filename)

            try:
                shutil.copy2(source_file_path, destination_file_path) # copy2 preserves metadata
                print(f"✅ {source_file_path} kopyalandı: {destination_file_path}")
            except FileNotFoundError:
                print(f"⚠️ {source_file_path} bulunamadı.")
            except Exception as e:
                print(f"❌ Hata: {source_file_path} kopyalanamadı. Hata: {e}")

    # Dosyalar için
    for file in files:
      source_file_path = os.path.join(root, file)
      destination_file_path = os.path.join(destination_dir, os.path.relpath(source_file_path, source_dir))

      try:
          shutil.copy2(source_file_path, destination_file_path) # copy2 preserves metadata
          print(f"✅ {source_file_path} kopyalandı: {destination_file_path}")
      except FileNotFoundError:
          print(f"⚠️ {source_file_path} bulunamadı.")
      except Exception as e:
          print(f"❌ Hata: {source_file_path} kopyalanamadı. Hata: {e}")


❌ Hata: /content/drive/MyDrive/wood_otsu/wood/train kopyalanamadı. Hata: [Errno 21] Is a directory: '/content/drive/MyDrive/wood_otsu/wood/train'
❌ Hata: /content/drive/MyDrive/wood_otsu/wood/test kopyalanamadı. Hata: [Errno 21] Is a directory: '/content/drive/MyDrive/wood_otsu/wood/test'
❌ Hata: /content/drive/MyDrive/wood_otsu/wood/ground_truth kopyalanamadı. Hata: [Errno 21] Is a directory: '/content/drive/MyDrive/wood_otsu/wood/ground_truth'
❌ Hata: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256 kopyalanamadı. Hata: [Errno 21] Is a directory: '/content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256'
❌ Hata: /content/drive/MyDrive/wood_otsu/wood/train/good kopyalanamadı. Hata: [Errno 21] Is a directory: '/content/drive/MyDrive/wood_otsu/wood/train/good'
❌ Hata: /content/drive/MyDrive/wood_otsu/wood/test/defect kopyalanamadı. Hata: [Errno 21] Is a directory: '/content/drive/MyDrive/wood_otsu/wood/test/defect'
❌ Hata: /content/drive/MyDrive/wood_otsu/wood/test/g

# Augmentation İşlemleri

In [ ]:
# Augmentation - 1
import os
from PIL import Image
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import random
import torch

# Görsel klasörü yolu (Google Drive içindeki)
image_dir = "/content/drive/MyDrive/wood_otsu_augmentation/wood/train/good"

# Uygulanacak augmentation'lar (örnek)
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
])

# Klasördeki her görseli işle
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
        filepath = os.path.join(image_dir, filename)

        # Görseli aç
        image = Image.open(filepath).convert('RGB')

        # Augmentation uygula
        augmented = augmentation_transforms(image)

        # Yeni isimle kaydet
        new_filename = f"aug_{filename}"
        new_filepath = os.path.join(image_dir, new_filename)
        augmented.save(new_filepath)

print("Tüm görseller augment edildi ve kaydedildi.")

In [19]:
# Augmentation - 2
import os
from PIL import Image
import torchvision.transforms as transforms
import random

# Görsellerin bulunduğu klasör
image_dir = "/content/drive/MyDrive/wood_otsu_augmentation/wood/train/good"

# 5 farklı augmentation varyasyonu için transform listesi
augmentation_variants = [
    transforms.Compose([
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.RandomRotation(degrees=15),
    ]),
    transforms.Compose([
        transforms.RandomVerticalFlip(p=1.0),
        transforms.ColorJitter(brightness=0.2),
    ]),
    transforms.Compose([
        transforms.RandomRotation(degrees=25),
        transforms.ColorJitter(contrast=0.3),
    ]),
    transforms.Compose([
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ColorJitter(saturation=0.4),
    ]),
    transforms.Compose([
        transforms.RandomRotation(degrees=(-30, 30)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    ])
]

# Görselleri işle
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
        filepath = os.path.join(image_dir, filename)
        image = Image.open(filepath).convert('RGB')

        for i, transform in enumerate(augmentation_variants):
            augmented = transform(image)
            new_filename = f"aug{i+1}_{filename}"
            new_filepath = os.path.join(image_dir, new_filename)
            augmented.save(new_filepath)

print("Tüm augmentasyonlar başarıyla oluşturuldu! 🎉")


Tüm görseller augment edildi ve kaydedildi.


In [27]:
# Augmentation - 3
import os
from PIL import Image, ImageFilter
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import random
import numpy as np
import torch

# Kaynak klasör
image_dir = "/content/drive/MyDrive/wood_otsu_augmentation/wood/train/good"

# Gaussian Noise fonksiyonu (torch tensor ile çalışır)
def add_gaussian_noise(img, mean=0., std=0.05):
    img_tensor = TF.to_tensor(img)
    noise = torch.randn(img_tensor.size()) * std + mean
    noisy = img_tensor + noise
    noisy = torch.clamp(noisy, 0., 1.)
    return TF.to_pil_image(noisy)

# Her resim için 5 farklı augmentasyon varyasyonu:
def get_augmented_images(image):
    return [
        # 1. Yatay çevirme
        TF.hflip(image),

        # 2. Hafif döndürme (±10°)
        TF.rotate(image, angle=random.uniform(-10, 10)),

        # 3. Parlaklık değişimi
        transforms.ColorJitter(brightness=0.2)(image),

        # 4. Gaussian Blur
        image.filter(ImageFilter.GaussianBlur(radius=1)),

        # 5. Gaussian Noise eklenmiş
        add_gaussian_noise(image)
    ]

# Görselleri işle ve augmentasyonları uygula
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
        filepath = os.path.join(image_dir, filename)
        image = Image.open(filepath).convert('RGB')

        augmented_images = get_augmented_images(image)

        for i, aug_img in enumerate(augmented_images):
            new_filename = f"aug{i+1}_{filename}"
            new_filepath = os.path.join(image_dir, new_filename)
            aug_img.save(new_filepath)

print("✅ Ahşap dostu augmentasyonlar tamamlandı!")

✅ Ahşap dostu augmentasyonlar tamamlandı!


# Kaç Dosya Var

In [28]:
import os

image_dir = "/content/drive/MyDrive/wood_otsu_augmentation/wood/train/good"
file_count = len([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
print(f"The directory '{image_dir}' contains {file_count} files.")


The directory '/content/drive/MyDrive/wood_otsu_augmentation/wood/train/good' contains 420 files.


# Eğitim Aşamaları

In [34]:
!python /content/drive/MyDrive/EfficientAD/efficientad.py \
  --mvtec_ad_path '/content/drive/MyDrive/wood_otsu' \
  --weights '/content/drive/MyDrive/EfficientAD/models/teacher_small.pth' \
  --output_dir '/content/drive/MyDrive/outputs/efficient' \
  --subdataset wood \
  --model_size small \
  --imagenet_train_path none \
  --train_steps 20000

Computing mean of features: 100% 63/63 [00:00<00:00, 136.12it/s]
Computing std of features: 100% 63/63 [00:00<00:00, 123.56it/s]
Current loss: 5.0175:  50% 9993/20000 [03:08<02:24, 69.24it/s]
Intermediate map normalization: 100% 7/7 [00:00<00:00, 141.30it/s]

Intermediate inference:   0% 0/140 [00:00<?, ?it/s]
Intermediate inference:  12% 17/140 [00:00<00:00, 165.79it/s]
Intermediate inference:  24% 34/140 [00:00<00:00, 124.57it/s]
Intermediate inference:  36% 51/140 [00:00<00:00, 140.82it/s]
Intermediate inference:  49% 68/140 [00:00<00:00, 150.53it/s]
Intermediate inference:  61% 85/140 [00:00<00:00, 155.18it/s]
Intermediate inference:  72% 101/140 [00:00<00:00, 150.57it/s]
Intermediate inference:  84% 118/140 [00:00<00:00, 154.24it/s]
Intermediate inference: 100% 140/140 [00:00<00:00, 152.77it/s]

✅ Anomaly Detection Sonucları
AUC Score          : 0.9171
F1 Score           : 0.8919
Precision          : 0.8462
Recall             : 0.9429
Optimal Threshold  : 0.2261
Confusion Matrix:


In [32]:
import os
import numpy as np
import cv2
import tifffile as tiff
from glob import glob
from sklearn.metrics import jaccard_score

# Yol tanımları
pred_mask_dir = '/content/drive/MyDrive/outputs/efficient/anomaly_maps/mvtec_ad/wood/test/defect'
gt_mask_dir = '/content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect'

# Dosya yollarını al
pred_mask_paths = sorted(glob(os.path.join(pred_mask_dir, '*.tiff')))
gt_mask_paths = sorted(glob(os.path.join(gt_mask_dir, '*_mask.jpg')))

ious = []

# Tahmin maskesi ve ground-truth maske eşlemesi
for pred_path in pred_mask_paths:
    # Dosya adı, örnek: 100000000.tiff -> 100000000_mask.jpg
    file_id = os.path.splitext(os.path.basename(pred_path))[0]
    gt_filename = f"{file_id}_mask.jpg"
    gt_path = os.path.join(gt_mask_dir, gt_filename)

    # Maskeleri yükle
    pred_mask = tiff.imread(pred_path)
    gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    # Maskeleri binary (0 ve 1) hale getir
    pred_bin = (pred_mask > 0.15).astype(np.uint8)
    gt_bin = (gt_mask > 0).astype(np.uint8)

    # Flatten ve IoU hesapla
    iou = jaccard_score(gt_bin.flatten(), pred_bin.flatten())
    ious.append(iou)

    print(f"{file_id} için IoU: {iou:.4f}")

# Ortalama IoU
if ious:
    mean_iou = np.mean(ious)
    print(f"\n🔍 Ortalama IoU: {mean_iou:.4f}")
else:
    print("⚠️ Hiç eşleşen maske bulunamadı.")

100000000 için IoU: 0.5792
100000003 için IoU: 0.2993
100000006 için IoU: 0.2809
100000007 için IoU: 0.6155
100000008 için IoU: 0.2753
100000009 için IoU: 0.0000
100000012 için IoU: 0.3166
100000014 için IoU: 0.5557
100000016 için IoU: 0.3223
100000018 için IoU: 0.4026
100000020 için IoU: 0.0000
100000021 için IoU: 0.4492
100000056 için IoU: 0.5185
100000069 için IoU: 0.1357
100000071 için IoU: 0.0279
100000073 için IoU: 0.4256
100100002 için IoU: 0.5613
100100003 için IoU: 0.5404
100100004 için IoU: 0.5069
100100005 için IoU: 0.3231
100100008 için IoU: 0.3919
100100026 için IoU: 0.5562
100100027 için IoU: 0.5181
100100033 için IoU: 0.5736
100100034 için IoU: 0.3374
100100035 için IoU: 0.2970
100100036 için IoU: 0.4259
100100037 için IoU: 0.3128
100100038 için IoU: 0.2593
100100039 için IoU: 0.3719
100100043 için IoU: 0.4212
100100048 için IoU: 0.3637
100100071 için IoU: 0.1155
100400000 için IoU: 0.3800
100500011 için IoU: 0.1840
100500053 için IoU: 0.0000
100700037 için IoU: 0.2061
1

# Augmentation Data İle
350 Sanal + 70 Gerçek Resim

In [29]:
!python /content/drive/MyDrive/EfficientAD/efficientad.py \
  --mvtec_ad_path '/content/drive/MyDrive/wood_otsu_augmentation' \
  --weights '/content/drive/MyDrive/EfficientAD/models/teacher_small.pth' \
  --output_dir '/content/drive/MyDrive/outputs/efficient' \
  --subdataset wood \
  --model_size small \
  --imagenet_train_path none \
  --train_steps 20000

Computing mean of features: 100% 378/378 [00:02<00:00, 130.72it/s]
Computing std of features: 100% 378/378 [00:01<00:00, 272.72it/s]
Current loss: 5.4128  :  50% 9998/20000 [02:11<02:14, 74.40it/s]
Intermediate map normalization:   0% 0/42 [00:00<?, ?it/s]
Intermediate map normalization:  29% 12/42 [00:00<00:00, 119.61it/s]
Intermediate map normalization:  62% 26/42 [00:00<00:00, 130.13it/s]
Intermediate map normalization: 100% 42/42 [00:00<00:00, 131.82it/s]

Intermediate inference:   0% 0/140 [00:00<?, ?it/s]
Intermediate inference:   6% 8/140 [00:00<00:01, 79.68it/s]
Intermediate inference:  18% 25/140 [00:00<00:00, 128.32it/s]
Intermediate inference:  30% 42/140 [00:00<00:00, 143.89it/s]
Intermediate inference:  41% 58/140 [00:00<00:00, 148.69it/s]
Intermediate inference:  52% 73/140 [00:00<00:00, 146.34it/s]
Intermediate inference:  64% 90/140 [00:00<00:00, 151.39it/s]
Intermediate inference:  76% 106/140 [00:00<00:00, 153.69it/s]
Intermediate inference:  88% 123/140 [00:00<00:00,

In [30]:
import os
import numpy as np
import cv2
import tifffile as tiff
from glob import glob
from sklearn.metrics import jaccard_score

# Yol tanımları
pred_mask_dir = '/content/drive/MyDrive/outputs/efficient/anomaly_maps/mvtec_ad/wood/test/defect'
gt_mask_dir = '/content/drive/MyDrive/wood_otsu_augmentation/wood/ground_truth_resized_256/defect'

# Dosya yollarını al
pred_mask_paths = sorted(glob(os.path.join(pred_mask_dir, '*.tiff')))
gt_mask_paths = sorted(glob(os.path.join(gt_mask_dir, '*_mask.jpg')))

ious = []

# Tahmin maskesi ve ground-truth maske eşlemesi
for pred_path in pred_mask_paths:
    # Dosya adı, örnek: 100000000.tiff -> 100000000_mask.jpg
    file_id = os.path.splitext(os.path.basename(pred_path))[0]
    gt_filename = f"{file_id}_mask.jpg"
    gt_path = os.path.join(gt_mask_dir, gt_filename)

    # Maskeleri yükle
    pred_mask = tiff.imread(pred_path)
    gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    # Maskeleri binary (0 ve 1) hale getir
    pred_bin = (pred_mask > 0.15).astype(np.uint8)
    gt_bin = (gt_mask > 0).astype(np.uint8)

    # Flatten ve IoU hesapla
    iou = jaccard_score(gt_bin.flatten(), pred_bin.flatten())
    ious.append(iou)

    print(f"{file_id} için IoU: {iou:.4f}")

# Ortalama IoU
if ious:
    mean_iou = np.mean(ious)
    print(f"\n🔍 Ortalama IoU: {mean_iou:.4f}")
else:
    print("⚠️ Hiç eşleşen maske bulunamadı.")

100000000 için IoU: 0.1501
100000003 için IoU: 0.0000
100000006 için IoU: 0.2391
100000007 için IoU: 0.1133
100000008 için IoU: 0.2418
100000009 için IoU: 0.0000
100000012 için IoU: 0.0000
100000014 için IoU: 0.4110
100000016 için IoU: 0.0753
100000018 için IoU: 0.1101
100000020 için IoU: 0.0000
100000021 için IoU: 0.1935
100000056 için IoU: 0.2689
100000069 için IoU: 0.0000
100000071 için IoU: 0.0000
100000073 için IoU: 0.4430
100100002 için IoU: 0.0084
100100003 için IoU: 0.0000
100100004 için IoU: 0.3039
100100005 için IoU: 0.0000
100100008 için IoU: 0.3920
100100026 için IoU: 0.1355
100100027 için IoU: 0.1225
100100033 için IoU: 0.1050
100100034 için IoU: 0.0000
100100035 için IoU: 0.2921
100100036 için IoU: 0.0340
100100037 için IoU: 0.2590
100100038 için IoU: 0.0865
100100039 için IoU: 0.0000
100100043 için IoU: 0.1391
100100048 için IoU: 0.2039
100100071 için IoU: 0.0000
100400000 için IoU: 0.0164
100500011 için IoU: 0.0179
100500053 için IoU: 0.0000
100700037 için IoU: 0.0000
1

# Augmentation İşlemi - 2
70 Sanal + 70 Gerçek Veri

In [21]:
!python /content/drive/MyDrive/EfficientAD/efficientad.py \
  --mvtec_ad_path '/content/drive/MyDrive/wood_otsu_augmentation' \
  --weights '/content/drive/MyDrive/EfficientAD/models/teacher_small.pth' \
  --output_dir '/content/drive/MyDrive/outputs/efficient' \
  --subdataset wood \
  --model_size small \
  --imagenet_train_path none \
  --train_steps 20000

Computing mean of features: 100% 126/126 [00:00<00:00, 196.58it/s]
Computing std of features: 100% 126/126 [00:00<00:00, 178.60it/s]
Current loss: 4.1606  :  50% 9997/20000 [02:26<02:15, 73.75it/s]
Intermediate map normalization:   0% 0/14 [00:00<?, ?it/s]
Intermediate map normalization: 100% 14/14 [00:00<00:00, 112.78it/s]

Intermediate inference:   0% 0/140 [00:00<?, ?it/s]
Intermediate inference:  12% 17/140 [00:00<00:00, 159.62it/s]
Intermediate inference:  24% 33/140 [00:00<00:00, 117.35it/s]
Intermediate inference:  35% 49/140 [00:00<00:00, 132.87it/s]
Intermediate inference:  47% 66/140 [00:00<00:00, 143.08it/s]
Intermediate inference:  59% 82/140 [00:00<00:00, 147.34it/s]
Intermediate inference:  70% 98/140 [00:00<00:00, 150.96it/s]
Intermediate inference:  81% 114/140 [00:00<00:00, 147.99it/s]
Intermediate inference: 100% 140/140 [00:00<00:00, 144.56it/s]

✅ Anomaly Detection Sonucları
AUC Score          : 0.9516
F1 Score           : 0.9189
Precision          : 0.8718
Recall  

In [25]:
import os
import numpy as np
import cv2
import tifffile as tiff
from glob import glob
from sklearn.metrics import jaccard_score

# Yol tanımları
pred_mask_dir = '/content/drive/MyDrive/outputs/efficient/anomaly_maps/mvtec_ad/wood/test/defect'
gt_mask_dir = '/content/drive/MyDrive/wood_otsu_augmentation/wood/ground_truth_resized_256/defect'

# Dosya yollarını al
pred_mask_paths = sorted(glob(os.path.join(pred_mask_dir, '*.tiff')))
gt_mask_paths = sorted(glob(os.path.join(gt_mask_dir, '*_mask.jpg')))

ious = []

# Tahmin maskesi ve ground-truth maske eşlemesi
for pred_path in pred_mask_paths:
    # Dosya adı, örnek: 100000000.tiff -> 100000000_mask.jpg
    file_id = os.path.splitext(os.path.basename(pred_path))[0]
    gt_filename = f"{file_id}_mask.jpg"
    gt_path = os.path.join(gt_mask_dir, gt_filename)

    # Maskeleri yükle
    pred_mask = tiff.imread(pred_path)
    gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    # Maskeleri binary (0 ve 1) hale getir
    pred_bin = (pred_mask > 0.15).astype(np.uint8)
    gt_bin = (gt_mask > 0).astype(np.uint8)

    # Flatten ve IoU hesapla
    iou = jaccard_score(gt_bin.flatten(), pred_bin.flatten())
    ious.append(iou)

    print(f"{file_id} için IoU: {iou:.4f}")

# Ortalama IoU
if ious:
    mean_iou = np.mean(ious)
    print(f"\n🔍 Ortalama IoU: {mean_iou:.4f}")
else:
    print("⚠️ Hiç eşleşen maske bulunamadı.")

100000000 için IoU: 0.5110
100000003 için IoU: 0.1779
100000006 için IoU: 0.3088
100000007 için IoU: 0.5742
100000008 için IoU: 0.2800
100000009 için IoU: 0.0000
100000012 için IoU: 0.2690
100000014 için IoU: 0.5842
100000016 için IoU: 0.2457
100000018 için IoU: 0.3439
100000020 için IoU: 0.0000
100000021 için IoU: 0.4464
100000056 için IoU: 0.5388
100000069 için IoU: 0.1163
100000071 için IoU: 0.0310
100000073 için IoU: 0.5105
100100002 için IoU: 0.5512
100100003 için IoU: 0.5143
100100004 için IoU: 0.6424
100100005 için IoU: 0.2366
100100008 için IoU: 0.4713
100100026 için IoU: 0.5539
100100027 için IoU: 0.5151
100100033 için IoU: 0.4955
100100034 için IoU: 0.3233
100100035 için IoU: 0.4182
100100036 için IoU: 0.3191
100100037 için IoU: 0.3924
100100038 için IoU: 0.1941
100100039 için IoU: 0.3035
100100043 için IoU: 0.3969
100100048 için IoU: 0.3244
100100071 için IoU: 0.0438
100400000 için IoU: 0.2799
100500011 için IoU: 0.1825
100500053 için IoU: 0.0000
100700037 için IoU: 0.1518
1